In [2]:
%pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [peft]━━━━━━ 1/2 [peft]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os 
import torch 
import json 
from datasets import load_dataset # 허깅 페이스나 대용량 데이터를 쉽게 불러온다.
from transformers import (
    AutoModelForCausalLM, # 언어 모델을 자동으로 불러온다.
    AutoTokenizer, # 모델에 맞는 토크나이저를 자동으로 불러온다
    BitsAndBytesConfig, # 양자화 설정을 위한 라이브러리
    TrainingArguments, # 모델 훈련에 필요한 모든 설정을 담는 라이블리
    Trainer, # 실제 훈련 과정을 관리 하는 메일 라이브러리 
    DataCollatorForLanguageModeling # 데이터를 배치 단위로 묶어준다.
)

# peft ( parameter-Efficient Fine Tuning ) LoRA와 관련된 설정
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [ ]:
MODEL_ID = ""
DATASET_PATH = ""
OUTPUT_DIR = ""

# MODEL에 맞는 tokenizer를 불러오고, 만일 해당 tokenizer에 padding 설정이 되어있지 않다면 , 이를 EOS 토큰으로 추가해준다.
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_model_code=True)
if tokenizer.pad_token is None : 
    tokenizer.pad_token = tokenizer.eos_token

# 아래 방식은 attention mask를 모든 token에 부여하여, 질문, 선택지, 답변에 대해서 모두 모델이 예측하도록 훈련하게 됩니다.

In [ ]:
def preprocess_function(examples) : 
    # prompt 템플릿 설정
    prompt_template = """
        다음 질문에 대한 옳은 답변을 선택지에서 고르세요.

        ### 질문 : 
        {question}

        ### 선택지 : 
        1. {option_1}
        2. {option_2}
        3. {option_3}
        4. {option_4}

        ### 답변 : 
    """

    # examples 라는 데이터셋에서 option과 question, answer를 추출한다.
    options = examples['options']
    formatted_options = {
        'option_1' : options.get('1',''), 'option_2' : options.get('2',''),
        'option_3' : options.get('3', ''), 'option_4' : options.get('4', '')
    }

    # **formatted_options 여기서 '**'는 dictionary unpacking을 의미한다.
    prompt = prompt_template.format(question=examples['question'], **formatted_options)
    full_text = prompt + str(examples['answer'])

    tokenized_output = tokenizer(
        full_text,
        truncation=True,
        max_length=1024,
        # padding="longest" 를 사용하면 배치 내 가장 긴 토큰으로 맞춰지게된다.
        padding='max_length'
    )
    
    """tokenized_output
    해당 변순의 경우 크게 2가지로 구성되어있습니다.
        input_ids : []  # 입력 prompt의 token들의 리스트
        attention_maks : [] # 실제 입력과 패딩을 구별하기 위한 리스트
    """


    tokenized_output["label"] = tokenized_output["input_ids"].copy()
    return tokenized_output



# answer 만을 기준으로 모델을 학습 시키는 코드 작성하기

In [ ]:
# 전처리 함수 (수정 및 개선 버전)
def preprocess_function_with_loss_mask(examples):
    # 프롬프트 템플릿 정의
    prompt_template = """
        다음 질문에 대한 옳은 답변을 선택지에서 고르세요.

        ### 질문 : 
        {question}

        ### 선택지 : 
        1. {option_1}
        2. {option_2}
        3. {option_3}
        4. {option_4}

        ### 답변 : 
    """
    
    # 옵션 포맷팅
    options = examples['options']
    formatted_options = {
        'option_1' : options.get('1', ''), 'option_2' : options.get('2', ''),
        'option_3' : options.get('3', ''), 'option_4' : options.get('4', '')
    }
    
    # 프롬프트와 답변 생성
    prompt = prompt_template.format(question=examples['question'], **formatted_options)
    answer = str(options['answer'])

    # 1. 프롬프트 부분의 길이를 정확히 계산 (BOS 토큰 등 포함)
    #    - 나중에 loss mask를 적용하기 위한 기준점
    prompt_len = len(tokenizer(prompt, add_special_tokens=True)["input_ids"])

    # 2. 프롬프트와 답변을 합쳐서 토큰화
    #    - 여기서는 아직 패딩이나 잘라내기를 하지 않습니다.
    full_text = prompt + answer
    #    - `add_special_tokens=True`는 보통 모델 입력 시작(BOS)과 끝(EOS)을 위해 필요합니다.
    #      모델에 따라 `False`로 설정해야 할 수도 있습니다.
    tokenized_full = tokenizer(full_text, add_special_tokens=True)
    
    input_ids = tokenized_full["input_ids"]
    
    # 3. 레이블 생성 및 loss mask 적용
    labels = input_ids.copy()
    labels[:prompt_len] = [-100] * prompt_len # 프롬프트 부분은 loss 계산에서 제외

    # 4. 토큰화된 결과와 원본 길이를 함께 반환
    #    - 나중에 filter에서 길이를 확인하기 위함
    tokenized_output = {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": tokenized_full["attention_mask"],
        "original_len": len(input_ids) # 길이를 체크하기 위한 임시 컬럼 추가
    }
    
    return tokenized_output

In [ ]:
if __name__ == "__main__" : 
    dataset = load_dataset('json', data_files=DATASET_PATH, split='train')

    dataset = dataset.map(preprocess_function_with_loss_mask, remove_columns=list(dataset.features))

    ############ token 길이가 1024가 넘는 경우 제거 #############
    def filter_long_data(example):
        return example['original_len'] <= 1024
    filtered_dataset = dataset.filter(filter_long_data)
    final_dataset = filtered_dataset.remove_columns(['original_len'])
    ############# ############# ############# ############# ############# #############


    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True, # 4비트 양자화
        bnb_4bit_use_double_quant=True, 
        bnb_4bit_quant_type="nf4", # 가중치 분포 최적화
        bnb_4bit_compute_dtype=torch.bfloat16 # 계산시 4비트를 16비트로 변환 -> 학습의 안정성
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config,
        device_map="auto"
    )
    model = prepare_model_for_kbit_training(model) # 양자화 모델의 안정성을 높혀주는 함수


    peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
        task_type="CASUAL_LM"
    )

    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()

    training_arguments = TrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        num_train_epochs=5,
        logging_steps=10,
        fp16=True,
        save_strategy="epoch",
        optim="paged_adamw_8bit"
    )

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    trainer = Trainer(
        model=model,
        args=training_arguments,
        train_dataset=dataset,
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    print('LoRA 파인튜닝 시작...')
    trainer.train()

    print(f"학습된 LoRA 어뎁터를 {OUTPUT_DIR}에 저장합니다. ")
    trainer.save_model(OUTPUT_DIR)

    print('Fine Tunning 종료..')

